# Capstone Project: The Battle of Neighbourhoods

### Table of Contents
This report is presented in the following parts as shown below: 
1. Introduction 
2. Data 
3. Methodology 
4. Result 
5. Discussion and Conclusion

### 1. Introduction
#### 1.1 Description of the Problem
   Los Angeles is the most populous city in California and the second most populous city in Unites States, after New York, City. The population is close to 4 million.
The city is known for its Mediterranean climate, ethnic diversity.
With all the skill set that I have gained in this course, I would like to find a good location to start an Indian Restaurant in Los Angeles City.

#### 1.2 Discussion of the Background
   Los Angeles has wide range of ethnicities and also people from all over the world come here for tourism. Finding a good place to fine dine is always a challenge in highly populous places. There are many Indian Restaurants, how ever my client would like to open a new branch in a neighborhood where the Asian population is more and also where there are not many Indian Restaurants.

#### 1.3 Target Audience
   Considering the diversity of LA, there is a high multicultural sense. The resolution to find the best location will help the tourists who love to eat highly authentic Indian food and also the people within the neighborhood.

## 2.Data 
   This project will rely on public data from Wikipedia, Foursquare and many other websites

#### 2.1.1 Dataset 1:
   For this project, the data related to the neighbourhoods in LA is pulled from:
       https://ehservices.publichealth.lacounty.gov/ezsearch
       http://www.laalmanac.com/communications/cm02_communities.php
   Rent inofrmation for each neighborhood is derived from: 
       https://www.rentcafe.com/average-rent-market-trends/us/ca/los-angeles/
   Population information for each neighborhood is derived from:
       http://maps.latimes.com/neighborhoods/ethnicity/asian/neighborhood/list/
       http://www.laalmanac.com/population/po24la.php
  
  The data is slice and diced and saved into csv files for further use in the analysis section of the project.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
# Get all the neighborhoods in LA City
git = 'https://raw.githubusercontent.com/pamancha/Coursera_Capstone/master/La_Data.csv'
la_data = pd.read_csv(git)
la_data

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00
1,Virgil Village,Los Angeles,CA,90004,1458,95.18
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47
3,Leimert Park,Los Angeles,CA,90008,1495,95.35
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00
8,Westwood,Los Angeles,CA,90024,2530,94.76
9,West Hollywood,Los Angeles,CA,90025,2400,93.34


In [6]:
#creating a dataframe for storing co-ordinates details.
coordinates = pd.DataFrame(columns=['Latitude','Longitude'])

# Using 'for loop' to get pass each Neighborhood name and get co-ordinates details through geocoding.
for row,neighborhood in la_data.iterrows():
    address = neighborhood['Neighborhood'] + ',' + neighborhood['City'] + ',' + neighborhood['State'] 
    try:
        geolocator = Nominatim(user_agent="my-application")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        # appending latitude and longitude values on coordiantes dataframe.
        coordinates = coordinates.append({'Latitude':latitude,'Longitude':longitude},ignore_index=True)
    except:
        print(address)

coordinates

Virgil Village,Los Angeles,CA
Dowtown Fashion District,Los Angeles,CA
Arlington Heights,Los Angeles,CA


,Latitude,Longitude
0,33.970090,-118.244994
1,34.072417,-118.288209
2,34.007702,-118.332063
3,34.045720,-118.252421
4,34.027234,-118.317576
5,34.056121,-118.430635
6,34.092301,-118.369289
7,34.074000,-118.260874
8,34.134427,-118.305092
9,34.081121,-118.177849


#### Resize LA neighborhood data and add both coordinates

In [8]:
La_Neighborhood = la_data.join(coordinates, how='outer')
La_Neighborhood

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47,34.007702,-118.332063
3,Leimert Park,Los Angeles,CA,90008,1495,95.35,34.045720,-118.252421
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72,34.027234,-118.317576
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48,34.056121,-118.430635
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50,34.092301,-118.369289
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00,34.074000,-118.260874
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092
9,West Hollywood,Los Angeles,CA,90025,2400,93.34,34.081121,-118.177849


In [9]:
# Max Rent as of 2019-June
maxrentLA = max(La_Neighborhood['Rent-2019-06'])
print('Max rent value of LA Neighborhood is $',maxrentLA)
# Min Rent as of 2019-June
minrentLA = min(La_Neighborhood['Rent-2019-06'])
print('Min rent value LA Neighborhood is $',minrentLA)

Max rent value of LA Neighborhood is $ 3195
Min rent value LA Neighborhood is $ 1246


In [10]:
#create a new dataframe to store rent score.
rent_Score = pd.DataFrame(columns=['RentScore'])
for index, Neighborhood in La_Neighborhood.iterrows():
    currentrentofN = Neighborhood['Rent-2019-06']
    rentscore =  (maxrentLA-currentrentofN)/(maxrentLA-minrentLA)
    rent_Score = rent_Score.append({'RentScore' : rentscore},ignore_index = True)
    
rent_Score

,RentScore
0,0.974859
1,0.891226
2,0.838892
3,0.872242
4,0.230888
5,0.356593
6,1.000000
7,0.847614
8,0.341201
9,0.407901


In [12]:
# Max Restaurant Rating as of 2019-June
maxRatingLA = max(La_Neighborhood['Resturant_Rating'])
print('Max Resturant_Rating value of LA Neighborhood is $',maxRatingLA)
# Min Restaurant Rating as of 2019-June
minRatingLA = min(La_Neighborhood['Resturant_Rating'])
print('Min Resturant_Rating value LA Neighborhood is $',minRatingLA)

Max Resturant_Rating value of LA Neighborhood is $ 96.3
Min Resturant_Rating value LA Neighborhood is $ 91.72


In [13]:
#create a new dataframe to store rent score.
Resturant_Rating_Score = pd.DataFrame(columns=['Resturant_Rating_Score'])
for index, Neighborhood in La_Neighborhood.iterrows():
    currentrentofN = Neighborhood['Resturant_Rating']
    RatingScore =  (maxRatingLA-currentrentofN)/(maxRatingLA-minRatingLA)
    Resturant_Rating_Score = Resturant_Rating_Score.append({'Resturant_Rating_Score' : RatingScore},ignore_index = True)
    
Resturant_Rating_Score

,Resturant_Rating_Score
0,0.938865
1,0.244541
2,0.617904
3,0.207424
4,1.000000
5,0.179039
6,0.611354
7,0.502183
8,0.336245
9,0.646288


In [14]:
#Resizing the dataframe to add rentscore
La_Neighborhood = La_Neighborhood.join(rent_Score, how='outer')
La_Neighborhood

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47,34.007702,-118.332063,0.838892
3,Leimert Park,Los Angeles,CA,90008,1495,95.35,34.045720,-118.252421,0.872242
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72,34.027234,-118.317576,0.230888
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48,34.056121,-118.430635,0.356593
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50,34.092301,-118.369289,1.000000
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00,34.074000,-118.260874,0.847614
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201
9,West Hollywood,Los Angeles,CA,90025,2400,93.34,34.081121,-118.177849,0.407901


In [15]:
# Resizing the dataframe to add Restaurant Rating Score
La_Neighborhood = La_Neighborhood.join(Resturant_Rating_Score, how='outer')

In [16]:
La_Neighborhood

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859,0.938865
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226,0.244541
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47,34.007702,-118.332063,0.838892,0.617904
3,Leimert Park,Los Angeles,CA,90008,1495,95.35,34.045720,-118.252421,0.872242,0.207424
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72,34.027234,-118.317576,0.230888,1.000000
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48,34.056121,-118.430635,0.356593,0.179039
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50,34.092301,-118.369289,1.000000,0.611354
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00,34.074000,-118.260874,0.847614,0.502183
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201,0.336245
9,West Hollywood,Los Angeles,CA,90025,2400,93.34,34.081121,-118.177849,0.407901,0.646288


In [17]:
# Adding the missing coordinates from Google maps for Reseda, North Hills and Sun Valley
La_Neighborhood.iloc[26, La_Neighborhood.columns.get_loc('Latitude')] = 34.2011
La_Neighborhood.iloc[27, La_Neighborhood.columns.get_loc('Latitude')] = 34.2358
La_Neighborhood.iloc[28, La_Neighborhood.columns.get_loc('Latitude')] = 34.2279
La_Neighborhood.iloc[26, La_Neighborhood.columns.get_loc('Longitude')] = -118.5361
La_Neighborhood.iloc[27, La_Neighborhood.columns.get_loc('Longitude')] = -118.4849
La_Neighborhood.iloc[28, La_Neighborhood.columns.get_loc('Longitude')] = -118.3813
La_Neighborhood

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859,0.938865
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226,0.244541
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47,34.007702,-118.332063,0.838892,0.617904
3,Leimert Park,Los Angeles,CA,90008,1495,95.35,34.045720,-118.252421,0.872242,0.207424
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72,34.027234,-118.317576,0.230888,1.000000
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48,34.056121,-118.430635,0.356593,0.179039
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50,34.092301,-118.369289,1.000000,0.611354
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00,34.074000,-118.260874,0.847614,0.502183
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201,0.336245
9,West Hollywood,Los Angeles,CA,90025,2400,93.34,34.081121,-118.177849,0.407901,0.646288


In [18]:
# create map of New York using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighborhood in zip(La_Neighborhood['Latitude'], La_Neighborhood['Longitude'], La_Neighborhood['City'], La_Neighborhood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

In [21]:
git_population = 'https://raw.githubusercontent.com/pamancha/Coursera_Capstone/master/Asian_population.csv'
la_population = pd.read_csv(git_population)
la_population

,Neighborhood,Asian_population%
0,Arleta,10.33
1,Arlington Heights,13.09
2,Atwater Village,21.54
3,Beverly Glen,6.70
4,Brentwood,8.56
5,Cypress Park,10.15
6,Downtown Historic Core,12.99
7,Dowtown Fashion District,4.63
8,Echo Park,20.86
9,El Sereno,10.62


In [57]:
LA_Dataset = pd.merge(La_Neighborhood, la_population, on='Neighborhood', how='outer')
LA_Dataset

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859,0.938865,49.57
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226,0.244541,23.91
2,Harvard Heights,Los Angeles,CA,90006,1560,93.47,34.007702,-118.332063,0.838892,0.617904,9.65
3,Leimert Park,Los Angeles,CA,90008,1495,95.35,34.045720,-118.252421,0.872242,0.207424,3.23
4,Dowtown Fashion District,Los Angeles,CA,90013,2745,91.72,34.027234,-118.317576,0.230888,1.000000,4.63
5,Downtown Historic Core,Los Angeles,CA,90014,2500,95.48,34.056121,-118.430635,0.356593,0.179039,12.99
6,Jefferson Park,Los Angeles,CA,90018,1246,93.50,34.092301,-118.369289,1.000000,0.611354,2.77
7,Arlington Heights,Los Angeles,CA,90019,1543,94.00,34.074000,-118.260874,0.847614,0.502183,13.09
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201,0.336245,25.86
9,West Hollywood,Los Angeles,CA,90025,2400,93.34,34.081121,-118.177849,0.407901,0.646288,20.16


# 3. Methodology

## 3.1 Data Exploration
    
#### 3.1.1 Get Asian Population data for each neighborhood in LA and pick the top 5 neighborhoods for Analysis
#### 3.1.2 Using Foursquare, explore the single Neighborhood among the 5 neighborhoods. Florence-Graham is used for this

In [22]:

LA_DS = LA_Dataset.sort_values(by=['Asian_population%'], ascending=False).head(5)
LA_DS

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859,0.938865,49.57
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201,0.336245,25.86
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226,0.244541,23.91
20,Playa Vista,Los Angeles,CA,90094,2100,95.33,34.261947,-118.351746,0.561827,0.211790,22.43
13,Atwater Village,Los Angeles,CA,90039,1800,96.30,34.052140,-118.474070,0.715752,0.000000,21.54


### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'K03DS2JNHZQHETU1B5TFKCLJ1ETOKULGHYOHTN01BDOHRMZQ' # your Foursquare ID
CLIENT_SECRET = '5F415O4MUKT3PJHNMQKHQWI1GY1J1SOVDP4FAREEM3KJANWW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K03DS2JNHZQHETU1B5TFKCLJ1ETOKULGHYOHTN01BDOHRMZQ
CLIENT_SECRET:5F415O4MUKT3PJHNMQKHQWI1GY1J1SOVDP4FAREEM3KJANWW


In [24]:
# Picking up one neighborhood to find the venus in Foursquare
LA_DS.loc[0, 'Neighborhood']

'Florence-Graham'

In [25]:
neighborhood_latitude = LA_DS.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LA_DS.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = LA_DS.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Florence-Graham are 33.9700896, -118.2449937.


In [26]:
# Find all the venues around Florence-Graham Area

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=K03DS2JNHZQHETU1B5TFKCLJ1ETOKULGHYOHTN01BDOHRMZQ&client_secret=5F415O4MUKT3PJHNMQKHQWI1GY1J1SOVDP4FAREEM3KJANWW&v=20180605&ll=33.9700896,-118.2449937&radius=500&limit=100'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0d58f4d9a6e6002be653fc'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 33.974589604500004,
    'lng': -118.23957775478354},
   'sw': {'lat': 33.9655895955, 'lng': -118.25040964521645}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9e0b3a8efaa14314c274b7',
       'name': 'Superior Grocers',
       'location': {'address': '7316 Compton Ave',
        'crossStreet': 'Florence',
        'lat': 33.97328043615116,
        'lng': -118.24707865857803,
        'labeledLatLngs': [{'label

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Superior Grocers,Grocery Store,33.973280,-118.247079
1,Bill's Drive In,Burger Joint,33.974500,-118.244225
2,Rite Aid,Pharmacy,33.974383,-118.246351
3,Birrieria Jalisco,Mexican Restaurant,33.969996,-118.247543
4,Florence Metro Station,Light Rail Station,33.973918,-118.243285


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


#### Create a function to repetiviely to explore the neighborhoods of remaning 4 places

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
La_venues = getNearbyVenues(names=LA_DS['Neighborhood'],
                                   latitudes=LA_DS['Latitude'],
                                   longitudes=LA_DS['Longitude']
                                  )


Florence-Graham
Westwood
Virgil Village
Playa Vista
Atwater Village


In [34]:
La_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Florence-Graham,33.97009,-118.244994,Superior Grocers,33.973280,-118.247079,Grocery Store
1,Florence-Graham,33.97009,-118.244994,Bill's Drive In,33.974500,-118.244225,Burger Joint
2,Florence-Graham,33.97009,-118.244994,Rite Aid,33.974383,-118.246351,Pharmacy
3,Florence-Graham,33.97009,-118.244994,Birrieria Jalisco,33.969996,-118.247543,Mexican Restaurant
4,Florence-Graham,33.97009,-118.244994,Florence Metro Station,33.973918,-118.243285,Light Rail Station


In [35]:
La_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Atwater Village,21,21,21,21,21,21
Florence-Graham,5,5,5,5,5,5
Playa Vista,1,1,1,1,1,1
Virgil Village,43,43,43,43,43,43
Westwood,5,5,5,5,5,5


In [36]:
print('There are {} uniques categories.'.format(len(La_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [37]:
# one hot encoding
LA_onehot = pd.get_dummies(La_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
LA_onehot['Neighborhood'] = La_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [LA_onehot.columns[-1]] + list(LA_onehot.columns[:-1])
LA_onehot = LA_onehot[fixed_columns]

LA_onehot.head()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Coffee Shop,Dance Studio,Dessert Shop,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fried Chicken Joint,Grocery Store,Gym / Fitness Center,Historic Site,Indian Restaurant,Italian Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pet Café,Pharmacy,Photography Studio,Pizza Place,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shoe Store,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Taco Place,Tea Room,Thai Restaurant,Video Store,Vietnamese Restaurant
0,Florence-Graham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Florence-Graham,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Florence-Graham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Florence-Graham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Florence-Graham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
LA_grouped = LA_onehot.groupby('Neighborhood').mean().reset_index()
LA_grouped

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Coffee Shop,Dance Studio,Dessert Shop,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fried Chicken Joint,Grocery Store,Gym / Fitness Center,Historic Site,Indian Restaurant,Italian Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pet Café,Pharmacy,Photography Studio,Pizza Place,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shoe Store,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Taco Place,Tea Room,Thai Restaurant,Video Store,Vietnamese Restaurant
0,Atwater Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.047619,0.047619,0.000000,0.095238,0.047619,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.047619,0.0,0.047619,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.047619,0.000000
1,Florence-Graham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,Playa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Virgil Village,0.046512,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.000000,0.000000,0.046512,0.023256,0.023256,0.000000,0.023256,0.023256,0.000000,0.023256,0.046512,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.046512,0.0,0.0,0.023256,0.046512,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.023256,0.046512,0.0,0.000000,0.023256,0.000000,0.023256,0.000000,0.023256,0.023256,0.0,0.046512,0.023256,0.023256
4,Westwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000


In [41]:
print("Before One-hot encoding:", LA_onehot.shape)
print("After One-hot encoding:", LA_grouped.shape)

Before One-hot encoding: (75, 53)
After One-hot encoding: (5, 53)


In [42]:
LA_onehot.shape

(75, 53)

In [43]:
LA_grouped.head()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Coffee Shop,Dance Studio,Dessert Shop,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fried Chicken Joint,Grocery Store,Gym / Fitness Center,Historic Site,Indian Restaurant,Italian Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pet Café,Pharmacy,Photography Studio,Pizza Place,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shoe Store,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Taco Place,Tea Room,Thai Restaurant,Video Store,Vietnamese Restaurant
0,Atwater Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.047619,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.047619,0.047619,0.000000,0.095238,0.047619,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.047619,0.000000,0.000000,0.047619,0.047619,0.000000,0.047619,0.047619,0.047619,0.0,0.047619,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.047619,0.000000
1,Florence-Graham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.200000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,Playa Vista,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Virgil Village,0.046512,0.023256,0.069767,0.023256,0.023256,0.023256,0.023256,0.000000,0.000000,0.046512,0.023256,0.023256,0.000000,0.023256,0.023256,0.000000,0.023256,0.046512,0.023256,0.023256,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256,0.046512,0.0,0.0,0.023256,0.046512,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.023256,0.046512,0.0,0.000000,0.023256,0.000000,0.023256,0.000000,0.023256,0.023256,0.0,0.046512,0.023256,0.023256
4,Westwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000


In [44]:
LA_new = LA_onehot["Indian Restaurant"] == 1

In [45]:
La_Indian = LA_onehot[LA_new]

In [46]:
La_Indian.head()

,Neighborhood,ATM,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bakery,Burger Joint,Bus Station,Café,Candy Store,Chinese Restaurant,Coffee Shop,Dance Studio,Dessert Shop,Dim Sum Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fried Chicken Joint,Grocery Store,Gym / Fitness Center,Historic Site,Indian Restaurant,Italian Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Mexican Restaurant,Park,Peruvian Restaurant,Pet Café,Pharmacy,Photography Studio,Pizza Place,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shoe Store,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Supermarket,Taco Place,Tea Room,Thai Restaurant,Video Store,Vietnamese Restaurant
29,Virgil Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Find the top 5 venues in each Neighborhood

In [121]:
num_top_venues = 5

for hood in LA_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = LA_grouped[LA_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Atwater Village----
                  venue  freq
0    Italian Restaurant  0.10
1        Farmers Market  0.05
2    Salon / Barbershop  0.05
3  Gym / Fitness Center  0.05
4         Grocery Store  0.05


----Florence-Graham----
                venue  freq
0  Light Rail Station   0.2
1  Mexican Restaurant   0.2
2        Burger Joint   0.2
3            Pharmacy   0.2
4       Grocery Store   0.2


----Playa Vista----
                venue  freq
0                Lake   1.0
1              Arcade   0.0
2  Light Rail Station   0.0
3              Lounge   0.0
4  Mexican Restaurant   0.0


----Virgil Village----
             venue  freq
0      Art Gallery  0.07
1              ATM  0.05
2      Candy Store  0.05
3  Thai Restaurant  0.05
4   Sandwich Place  0.05


----Westwood----
                venue  freq
0                Park   0.4
1      Scenic Lookout   0.4
2            Tea Room   0.2
3                 ATM   0.0
4  Salon / Barbershop   0.0




In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
LA_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
LA_neighborhoods_venues_sorted['Neighborhood'] = LA_grouped['Neighborhood']

for ind in np.arange(LA_grouped.shape[0]):
    LA_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(LA_grouped.iloc[ind, :], num_top_venues)

LA_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Atwater Village,Italian Restaurant,Photography Studio,Salad Place,Gym / Fitness Center,Historic Site,Juice Bar,Video Store,Farmers Market,Dance Studio,Coffee Shop
1,Florence-Graham,Light Rail Station,Grocery Store,Mexican Restaurant,Burger Joint,Pharmacy,Vietnamese Restaurant,Dessert Shop,Historic Site,Gym / Fitness Center,Fried Chicken Joint
2,Playa Vista,Lake,Vietnamese Restaurant,Juice Bar,Indian Restaurant,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant,Fast Food Restaurant
3,Virgil Village,Art Gallery,ATM,Candy Store,Filipino Restaurant,Sandwich Place,Korean Restaurant,Mexican Restaurant,Thai Restaurant,Bakery,Burger Joint
4,Westwood,Park,Scenic Lookout,Tea Room,Vietnamese Restaurant,Dance Studio,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant


#### Using K-mean Clustering to identify the best cluster for opening the Indian Restaurant

In [50]:
# set number of clusters
kclusters = 3

LA_grouped_clustering = LA_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 1, 0], dtype=int32)

In [51]:
# add clustering labelsed
#del LA_neighborhoods_venues_sorted
LA_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [52]:

LA_merged = LA_DS

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
LA_merged = LA_merged.join(LA_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

LA_merged

,Neighborhood,City,State,ZipCode,Rent-2019-06,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Florence-Graham,Los Angeles,CA,90001,1295,92.00,33.970090,-118.244994,0.974859,0.938865,49.57,1,Light Rail Station,Grocery Store,Mexican Restaurant,Burger Joint,Pharmacy,Vietnamese Restaurant,Dessert Shop,Historic Site,Gym / Fitness Center,Fried Chicken Joint
8,Westwood,Los Angeles,CA,90024,2530,94.76,34.134427,-118.305092,0.341201,0.336245,25.86,0,Park,Scenic Lookout,Tea Room,Vietnamese Restaurant,Dance Studio,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant
1,Virgil Village,Los Angeles,CA,90004,1458,95.18,34.072417,-118.288209,0.891226,0.244541,23.91,1,Art Gallery,ATM,Candy Store,Filipino Restaurant,Sandwich Place,Korean Restaurant,Mexican Restaurant,Thai Restaurant,Bakery,Burger Joint
20,Playa Vista,Los Angeles,CA,90094,2100,95.33,34.261947,-118.351746,0.561827,0.211790,22.43,2,Lake,Vietnamese Restaurant,Juice Bar,Indian Restaurant,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant,Fast Food Restaurant
13,Atwater Village,Los Angeles,CA,90039,1800,96.30,34.052140,-118.474070,0.715752,0.000000,21.54,1,Italian Restaurant,Photography Studio,Salad Place,Gym / Fitness Center,Historic Site,Juice Bar,Video Store,Farmers Market,Dance Studio,Coffee Shop


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(LA_merged['Latitude'], LA_merged['Longitude'], LA_merged['Neighborhood'], LA_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
LA_merged.loc[LA_merged['Cluster Labels'] == 0, LA_merged.columns[[0] + list(range(5, LA_merged.shape[1]))]]

,Neighborhood,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Westwood,94.76,34.134427,-118.305092,0.341201,0.336245,25.86,0,Park,Scenic Lookout,Tea Room,Vietnamese Restaurant,Dance Studio,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant


In [55]:
LA_merged.loc[LA_merged['Cluster Labels'] == 1, LA_merged.columns[[0] + list(range(5, LA_merged.shape[1]))]]

,Neighborhood,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Florence-Graham,92.00,33.970090,-118.244994,0.974859,0.938865,49.57,1,Light Rail Station,Grocery Store,Mexican Restaurant,Burger Joint,Pharmacy,Vietnamese Restaurant,Dessert Shop,Historic Site,Gym / Fitness Center,Fried Chicken Joint
1,Virgil Village,95.18,34.072417,-118.288209,0.891226,0.244541,23.91,1,Art Gallery,ATM,Candy Store,Filipino Restaurant,Sandwich Place,Korean Restaurant,Mexican Restaurant,Thai Restaurant,Bakery,Burger Joint
13,Atwater Village,96.30,34.052140,-118.474070,0.715752,0.000000,21.54,1,Italian Restaurant,Photography Studio,Salad Place,Gym / Fitness Center,Historic Site,Juice Bar,Video Store,Farmers Market,Dance Studio,Coffee Shop


In [56]:
LA_merged.loc[LA_merged['Cluster Labels'] == 2, LA_merged.columns[[0] + list(range(5, LA_merged.shape[1]))]]

,Neighborhood,Resturant_Rating,Latitude,Longitude,RentScore,Resturant_Rating_Score,Asian_population%,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Playa Vista,95.33,34.261947,-118.351746,0.561827,0.21179,22.43,2,Lake,Vietnamese Restaurant,Juice Bar,Indian Restaurant,Historic Site,Gym / Fitness Center,Grocery Store,Fried Chicken Joint,Filipino Restaurant,Fast Food Restaurant


# 4. Result

The following are the highlights of the 3 clusters above:

Cluster 1 with Westwood pridominemntly appears to be the best location to open the new Restaurant.
Westwood seems to have park, scenic lookout as the most common venues and there is no Indian Restaurant,
Westwood also has second highest Asian population and the property rental is low and also there are no high rating restaurants in the area.

Cluster 2: The Renting is very high for Florence and Virgil Village already has Indian Restaurant.
Cluster 3: Playa Vista already has Indian Restaurant and it is already in most commone venues.


# 5. Discussion and Conclusion

   It is very important to note that Cluster 1 is more viable to open a new Indian Restaurant. The place seems to be more centric for tourists as park and site seeing are most common viewed places and also the rent is reasonably low in the neighborhood with no existing India Restuarants.